In [ ]:
!nvidia-smi

In [ ]:
import gdown
import zipfile
import os

In [ ]:
try:
    file_id = "https://drive.goo#gle.com/file/d/1n_0za90fHWo9uopP9XipKNcjLPvgrNT4/view?usp=sharing".split("/")[-2]
    prefix =  'https://drive.google.com/uc?/export=download&id='
    gdown.download(prefix + file_id)
except Exception as e:
    raise e

In [ ]:
with zipfile.ZipFile('/content/track-master.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import ntpath
import random
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Convolution2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa


In [ ]:
data_dir = '/content/data/track-master'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', None)
data.head()

In [ ]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail

data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head()


In [ ]:
print("shape of the dataset" , data.shape)
print("information about the dataset", data.info())


In [ ]:
data.describe()

# Doing EDA Part

In [ ]:
# analize the steering position
num_bins = 25
samples_per_bin = 400
his, bins = np.histogram(data["steering"],num_bins)
center = (bins[:-1]+ bins[1:]) * 0.5
bar_width = bins[1] - bins[0]
plt.bar(center, his, width=bar_width)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin),color="red")
plt.xlabel("Steering")
plt.ylabel("Frequency")
plt.title("Histogram of Steering Angles")
plt.show()

In [ ]:
remove_list = []
for j in range(num_bins):
    list_ = []
    for i in range(len(data['steering'])):
        if data['steering'][i] >= bins[j] and data['steering'][i] <= bins[j+1]:
            list_.append(i)
    list_ = shuffle(list_)
    list_ = list_[samples_per_bin:]
    remove_list.extend(list_)

print('removed', len(remove_list))
data.drop(data.index[remove_list], inplace=True)
print('remaining', len(data))

In [ ]:
his, bins = np.histogram(data["steering"],num_bins)
center = (bins[:-1]+ bins[1:]) * 0.5
bar_width = bins[1] - bins[0]
plt.bar(center, his, width=bar_width,)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin),color="red")
plt.xlabel("Steering")
plt.ylabel("Frequency")
plt.title("Histogram of Steering Angles")
plt.show()

In [ ]:
def load_img_steering(datadir, data):
  image_path = []
  steering = []
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip()))
    steering.append(float(indexed_data[3]))

    image_path.append(os.path.join(datadir, left.strip()))
    steering.append(float(indexed_data[3]) + 0.15)

    image_path.append(os.path.join(datadir, right.strip()))
    steering.append(float(indexed_data[3]) - 0.15)

  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings


In [ ]:
image_paths, steerings = load_img_steering(data_dir + '/IMG', data)

In [ ]:
image_paths

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(image_paths, steerings, test_size=0.2, random_state=40)
print('Training Samples: {}\nValid Samples: {}'.format(len(x_train), len(x_val)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(y_train, bins=num_bins, width=bar_width, color='green')
axes[0].set_title('Training Set')
axes[1].hist(y_val, bins=num_bins, width=bar_width, color='red')
axes[1].set_title('Validation Set')
plt.show()

In [ ]:
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.3))
  image = zoom.augment_image(image)
  return image

def flip(image, steering):
  image = cv2.flip(image, 1)
  steering = -steering
  return image, steering

def pan(image):
  pan = iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

def adjust_brightness(image):
  brightness = iaa.Multiply((0.4, 1.2))
  image = brightness.augment_image(image)
  return image


In [ ]:
def image_agmentation(image, steering):
    image = cv2.imread(image)
    if np.random.rand() < 0.5:
        image = zoom(image)
    if np.random.rand() < 0.5:
        image = pan(image)
    if np.random.rand() < 0.5:
        image = adjust_brightness(image)
    if np.random.rand() < 0.5:
        image, steering = flip(image, steering)
    return image, steering

In [ ]:
row = 5
col = 2
fig, axes = plt.subplots(row, col, figsize=(12,24))

for i in range (5):
  rand_num = random.randint(0, len(image_paths) - 1)
  image = image_paths[rand_num]
  steering = steerings[rand_num]

  original_image = cv2.imread(image)
  augmented_image, steering = image_agmentation(image, steering)

  axes[i][0].imshow(original_image)
  axes[i][0].set_title("Original Image")

  axes[i][1].imshow(augmented_image)
  axes[i][1].set_title("Augmented Image")

In [ ]:
def image_preprocess(img):
    img = img[50:300,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [ ]:
img = image_paths[100]
original_image = cv2.imread(img)
preprocessed_image = image_preprocess(original_image)

fig, axes = plt.subplots(1,2 , figsize=(12,10))
fig.tight_layout()
axes[0].imshow(original_image)
axes[0].set_title('Original Image')
axes[1].imshow(preprocessed_image)
axes[1].set_title('Preprocessed Image')



In [ ]:
def batch_generator(image_paths, steering_angle, batch_size, is_training):
  while True:
    batch_img = []
    batch_steering = []

    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)
      if is_training:
        image , steering = image_agmentation(image_paths[random_index], steering_angle[random_index])
      else:
        image = cv2.imread(image_paths[random_index])
        steering = steering_angle[random_index]

      image_pre = image_preprocess(image)
      batch_img.append(image_pre)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

In [ ]:
x_train_gen, y_train_gen = next(batch_generator(x_train, y_train, 1, 1))
x_val_gen, y_val_gen = next(batch_generator(x_val, y_val, 1,1))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.tight_layout()
axes[0].imshow(x_train_gen[0])
axes[0].set_title('Training Image')
axes[1].imshow(x_val_gen[0])
axes[1].set_title('Validation Image')

In [ ]:
# Modified NVIDIA Model
def nvidia_model():
    model = Sequential()

    model.add(Conv2D(24, (5, 5), strides=(2, 2), activation='elu', input_shape= (66, 200, 3)))
    model.add(Conv2D(36, (5, 5), strides=(2, 2),activation='elu'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2),activation='elu'))

    # 3x3 Convolutional layers with stride of 1x1
    model.add(Conv2D(64, (3, 3),activation='elu'))
    model.add(Conv2D(64, (3, 3),activation='elu'))

    # Flatten before passing to the fully connected layers
    model.add(Flatten())
    # Three fully connected layers
    model.add(Dense(100,activation='elu'))
    model.add(Dropout(.04))
    model.add(Dense(50,activation='elu'))
    model.add(Dropout(.04))
    model.add(Dense(10,activation='elu'))
    model.add(Dropout(.04))
    model.add(Dense(1))

    # Optimizer with reduced learning rate
    optimizer = Adam(learning_rate=1e-3)

    # Compile the model with proper regression metrics
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae'])

    return model


In [ ]:
model = nvidia_model()
print(model.summary())

In [ ]:
history = model.fit(batch_generator(x_train, y_train, 100, 1),
                    steps_per_epoch=300,
                    epochs=50,
                    validation_data=batch_generator(x_val, y_val, 100, 0),
                    validation_steps=200,
                    verbose=1,
                    shuffle=1)

In [ ]:
# Check available keys to avoid KeyErrors
print(history.history.keys())

plt.figure(figsize=(12, 5))

# Plot MSE Scores
plt.subplot(1, 2, 1)
plt.plot(history.history.get('mse', history.history.get('mean_squared_error')), label='Train MSE')
plt.plot(history.history.get('val_mse', history.history.get('val_mean_squared_error')), label='Validation MSE')
plt.title('MSE Score')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
plt.savefig('history.png')

In [ ]:
model.save('model.h5')